In [1]:
# Installing all the software tools we need:
# - transformers, datasets, accelerate: For working with LLMs
# - peft, bitsandbytes: For efficient memory training (QLoRA)
!pip install -q transformers datasets accelerate peft bitsandbytes trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 kB 18.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, DatasetDict

# --- Load a new, reliable instruction dataset (we'll use 'databricks-dolly-15k') ---
# This dataset is already formatted with 'instruction' and 'response' columns,
# making it perfect for fine-tuning! We are loading a smaller version for quick training.
print("Loading the Databricks Dolly 15k dataset...")

# Load the full dataset first
dolly_data = load_dataset("databricks/databricks-dolly-15k", split="train")

# Filter the data to keep only useful instructions (like writing tasks)
# and take only the first 1000 examples for speed.
dolly_data = dolly_data.filter(lambda x: x["category"] in ["creative_writing", "summarization", "open_qa"])
train_data_subset = dolly_data.select(range(1000))

# --- Format the Data for the LLM ---
def format_data_for_llm(example):
    # This creates the INSTRUCTION/RESPONSE format required by the model:
    # [INST] Write a short story about... [/INST] The story begins: 'Once upon a time...'
    return {
        "text": f"[INST] {example['instruction']} [/INST] {example['response']}"
    }

# Apply the formatting to the subset
formatted_data = train_data_subset.map(format_data_for_llm)

print("Dataset is prepared and ready!")
# Show the first example to see the format:
print("\nExample Story Format:")
print(formatted_data[0]['text'])

Loading the Databricks Dolly 15k dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15011 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset is prepared and ready!

Example Story Format:
[INST] Why can camels survive for long without water? [/INST] Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time.


In [3]:
# Split the dataset into 90% for training and 10% for testing
# We use 'formatted_data' now, which is the variable we created in the step above!
data_split = formatted_data.train_test_split(test_size=0.1)

train_data = data_split["train"]
test_data = data_split["test"]

print(f"Total training examples: {len(train_data)}")
print(f"Total testing examples: {len(test_data)}")

Total training examples: 900
Total testing examples: 100


In [4]:
from huggingface_hub import login

# When you run this, a pop-up window will ask for your Hugging Face Token.
# Paste the token (the long string you just copied) into the box and press Enter.
login()

print("Successfully logged into Hugging Face!")

Successfully logged into Hugging Face!


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# 1. Configuration for memory saving (QLoRA)
# This lets us train the giant model on the free GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Load it in 4-bit to save space
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

MODEL_ID = "google/gemma-2b-it"

# 2. Load the Model (the brain)
print(f"Loading Model: {MODEL_ID}. This may take a few minutes...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto" # This automatically puts the model on the GPU
)
# 3. Load the Tokenizer (the translator)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
print("Model and Tokenizer loaded successfully.")

Loading Model: google/gemma-2b-it. This may take a few minutes...


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Model and Tokenizer loaded successfully.


In [9]:
from peft import LoraConfig
from trl import SFTTrainer
from transformers import TrainingArguments

# 1. LORA Configuration (Your settings are correct here)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 2. Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1, # This must remain 1
    # === FIX IS HERE: Add Gradient Accumulation ===
    gradient_accumulation_steps=4,
    # ==============================================
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    fp16=False, # Changed to False
    bf16=True,  # Added bf16=True
)

# Define a Formatting Function (This is correct)
def formatting_function(examples):
    # This function tells the SFTTrainer exactly how to find the text column.
    return examples['text']

# 3. Set up the SFT Trainer
# We are only keeping the arguments that are absolutely required.
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    peft_config=lora_config,
    formatting_func=formatting_function,
    # REMOVED: tokenizer=tokenizer (because of the previous error)
    # REMOVED: max_seq_length=512 (because of the new error)
)

print("Training setup is complete. Ready to bake!")

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Training setup is complete. Ready to bake!


In [10]:
print("Starting the fine-tuning process...")
# This is the step that actually uses the GPU for training!
trainer.train()
print("Training is complete! The model has learned new story skills.")

Starting the fine-tuning process...


Step,Training Loss
10,3.704700
20,2.693300
30,2.469800
40,2.523200
50,2.098800
60,2.363700
70,2.067400
80,2.233100
90,2.270300
100,2.285800


wandb: WARNING URL not available in offline run
wandb: WARNING URL not available in offline run
wandb: WARNING URL not available in offline run


Training is complete! The model has learned new story skills.


In [11]:
from transformers import pipeline
import torch
import time

# Create a text generation pipeline
# We must include the tokenizer and set torch_dtype
story_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
)

# --- USER INPUT AREA ---
# 1. Define your story request here.
#    The variable 'prompt_text' is what your professor will ask for.
prompt_text = "Write a story about a brave astronaut cat who visits the moon."

# 2. Format the request with the tags the model was trained on.
formatted_prompt = f"[INST] {prompt_text} [/INST]"
# -----------------------

# Set up generation parameters
generation_args = {
    "max_new_tokens": 300,
    "do_sample": True,
    "temperature": 0.8,
    "top_k": 50,
    # Stop the generation immediately after the model's response is complete
    "eos_token_id": tokenizer.encode('[/INST]')[0],
}

print(f"\nYour request: {prompt_text}")

# Generate the story
# Using torch.no_grad() speeds up generation by disabling training calculations
with torch.no_grad():
    result = story_generator(
        formatted_prompt,
        **generation_args
    )

# Clean the Output for Presentation
generated_text = result[0]['generated_text']

# Find the end of your input prompt and take only the model's response part
if "[/INST]" in generated_text:
    final_story = generated_text.split("[/INST]")[1].strip()
else:
    final_story = generated_text

print("\n--- GENERATED STORY ---")
print(final_story)
print("-----------------------")

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0



Your request: Write a story about a brave astronaut cat who visits the moon.

--- GENERATED STORY ---
In the heart of the galaxy, there lived a courageous cat named Fido. While dogs tend to be large and strong, cats were known to be agile and acrobatic. One day, Fido decided to go on an adventure and visit the moon. 

Fido's journey was long and treacherous. He had to cross the galaxy in a rocket ship. It was a very challenging journey, but Fido persevered. Finally, he arrived on the moon and landed in the lunar city.

After exploring the lunar city, Fido decided to return home. He was a brave cat and had a good time. Fido flew back home for an exciting visit with his human friends.
-----------------------


In [12]:
# Save the small learned parts (the LoRA adapter weights)
print("Saving LoRA adapter weights...")
# This saves the model's new skills into a folder on the Colab server.
trainer.model.save_pretrained("./fine_tuned_story_model")
print("Adapter saved successfully to: ./fine_tuned_story_model")

Saving LoRA adapter weights...
Adapter saved successfully to: ./fine_tuned_story_model


In [ ]:
# Create a zip file of your trained model adapters
!zip -r story_model_adapters.zip fine_tuned_story_model/
print("Model folder has been compressed into 'story_model_adapters.zip'.")


  adding: fine_tuned_story_model/ (stored 0%)
  adding: fine_tuned_story_model/adapter_config.json (deflated 58%)
  adding: fine_tuned_story_model/adapter_model.safetensors (deflated 8%)
  adding: fine_tuned_story_model/README.md (deflated 65%)
Model folder has been compressed into 'story_model_adapters.zip'.


In [13]:
# --- 1. INSTALL GRADIO AND OTHER LIBS ---
# This ensures Gradio is installed in the new session.
!pip install -q gradio

# --- 2. IMPORT ALL NECESSARY LIBRARIES ---
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch
import os

# --- 3. GLOBAL SETUP (Load Model and Tokenizer ONLY ONCE) ---
# IMPORTANT: This path must match the folder you created on GitHub!
MODEL_ID = "google/gemma-2b-it"
LORA_PATH = "./fine_tuned_story_model"

print("Setting up QLoRA configuration for memory efficiency...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Function to handle model loading
def load_model_and_tokenizer():
    global model, tokenizer, story_generator

    # Load the compressed base model (Gemma 2B)
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto"
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    tokenizer.pad_token = tokenizer.eos_token

    # Attach your custom storytelling skills (PEFT adapters from the uploaded files)
    model = PeftModel.from_pretrained(base_model, LORA_PATH)
    model.eval()

    # Create the generation pipeline
    story_generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
    )
    return "Model Loaded Successfully!"

# --- 4. THE CORE GENERATION FUNCTION ---
# This function runs every time you click 'Submit' in the web app
def generate_story_app(prompt_text, max_length, creativity):
    if 'story_generator' not in globals():
        return "Model is not loaded. Please ensure the loading steps ran."

    # Format the Input with the required tags [INST] and [/INST]
    formatted_prompt = f"[INST] {prompt_text} [/INST]"

    generation_args = {
        "max_new_tokens": max_length,
        "do_sample": True,
        "temperature": creativity,
        "top_k": 50,
        "eos_token_id": tokenizer.encode('[/INST]')[0],
    }

    # Generate the story
    with torch.no_grad():
        result = story_generator(
            formatted_prompt,
            **generation_args
        )

    # Clean the Output (Strip the tags for presentation)
    generated_text = result[0]['generated_text']
    if "[/INST]" in generated_text:
        final_story = generated_text.split("[/INST]")[1].strip()
    else:
        final_story = generated_text

    return final_story

# --- 5. BUILD AND LAUNCH THE GRADIO INTERFACE ---

# Load model when the app starts
load_status = load_model_and_tokenizer()

# Define the visual components (boxes, sliders)
prompt_box = gr.Textbox(
    label="Your Story Prompt",
    placeholder="Type your request here...",
    lines=3
)
length_slider = gr.Slider(
    minimum=100, maximum=500, step=10, value=250,
    label="Story Length (Tokens)"
)
temp_slider = gr.Slider(
    minimum=0.1, maximum=1.0, step=0.1, value=0.8,
    label="Creativity / Temperature"
)

# Build the main interface
iface = gr.Interface(
    fn=generate_story_app,
    inputs=[prompt_box, length_slider, temp_slider],
    outputs=gr.Textbox(label="Generated Fairy Tale", lines=10),
    title="📚 Master's Project: Fine-Tuned Storytelling LLM",
    description=f"Model Status: {load_status}. Enter a prompt to generate a custom fairy tale. (Base Model: {MODEL_ID} + PEFT Adapters)",
    live=False
)

# Launch the app
iface.launch(share=True)

Setting up QLoRA configuration for memory efficiency...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed5b1176a8d6014a06.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
